
# Ensemble of Ensembles

## Ensemble of Ensembles

Ensemble of Ensembles is a meta-ensemble method that combines multiple ensemble methods to improve predictive performance. It leverages the strengths of different ensemble techniques, such as bagging, boosting, and stacking, to create a more robust model. The idea is to use the predictions from various ensemble methods as inputs to a final model, which can be another ensemble method or a simple model like logistic regression.

## Advantages of Ensemble of Ensembles

-   **Improved Accuracy**: By combining predictions from multiple ensemble methods, the overall accuracy can be significantly improved.
-   **Robustness**: It reduces the risk of overfitting by averaging out the predictions from different models, leading to better generalization.
-   **Flexibility**: It allows the use of diverse ensemble methods, which can capture different patterns in the data.
-   **Diversity**: The combination of different ensemble methods can lead to a more diverse set of predictions, which is beneficial for model performance.

## Disadvantages of Ensemble of Ensembles

-   **Complexity**: The model can become complex and harder to interpret, as it involves multiple layers of models.
-   **Computational Cost**: Training multiple ensemble methods can be computationally expensive and time-consuming.
-   **Risk of Overfitting**: If not managed properly, combining too many models can lead to overfitting, especially if the base models are too complex.
-   **Hyperparameter Tuning**: Each ensemble method may require its own hyperparameter tuning, adding to the complexity of the model development process.

## Types of Ensemble of Ensembles

Ensemble of Ensembles can be implemented using various ensemble methods, including:

-   **Stacking**: Combines predictions from multiple base models using a meta-model that learns how to best combine their outputs (e.g., logistic regression, random forest).
-   **Multi-level Stacking**: A more complex version of stacking where multiple layers of models are trained, and predictions from one layer are used as inputs to the next layer.

### Example: Stacking

In [ ]:
from sklearn import datasets

titanic = datasets.fetch_openml('titanic', version=1, as_frame=True)
df = titanic.frame[['pclass', 'sex', 'age', 'survived']]
X = df.drop(columns=['survived'])
y = df['survived'].astype('category').cat.codes
class_names = ['Not Survived', 'Survived']
feature_names = X.columns.tolist()
print(f"Feature names: {feature_names}")
print(f"Class labels: {class_names}")

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

preprocessing_pipeline = ColumnTransformer(
    transformers=[
        ('age', Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())
            ]
        ), ['age']),
        ('sex', Pipeline(
            steps=[
                ('encoder', OneHotEncoder(drop='first', sparse_output=False))
            ]
        ), ['sex']),
        ('pclass', Pipeline(
            steps=[
                ('enconder', OneHotEncoder(sparse_output=False))
            ]
        ), ['pclass'])
    ],
    remainder='passthrough'
)
X_train_transformed = preprocessing_pipeline.fit_transform(X_train)
X_test_transformed = preprocessing_pipeline.transform(X_test)

feature_names = preprocessing_pipeline.get_feature_names_out()
print(f"Transformed feature names: {feature_names}")

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
# Define base models
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('gb', GradientBoostingClassifier(n_estimators=100))
]
# Define the final model
final_model = LogisticRegression()
# Create the stacking ensemble
stacking_ensemble = StackingClassifier(estimators=base_models, final_estimator=final_model)
# Fit the stacking ensemble
stacking_ensemble.fit(X_train_transformed, y_train)

In [ ]:
# Evaluate the stacking ensemble
accuracy = stacking_ensemble.score(X_test_transformed, y_test)
print(f"Stacking Ensemble Accuracy: {accuracy:.2f}")
# Predict using the stacking ensemble
y_pred = stacking_ensemble.predict(X_test_transformed)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=class_names))

### Example: Multi-level Ensemble of Ensembles

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

# Define base models for the first level
base_models_level_1 = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('gb', GradientBoostingClassifier(n_estimators=100)),
    ('svc', SVC(probability=True))
]
# Create the first level ensemble using voting
first_level_ensemble = VotingClassifier(estimators=base_models_level_1, voting='soft')
# Fit the first level ensemble
first_level_ensemble.fit(X_train_transformed, y_train)

# Define base models for the second level
base_models_level_2 = [
    ('first_level', first_level_ensemble),
    ('log_reg', LogisticRegression())
]
# Create the second level ensemble using stacking
second_level_ensemble = StackingClassifier(estimators=base_models_level_2, final_estimator=LogisticRegression())
# Fit the second level ensemble
second_level_ensemble.fit(X_train_transformed, y_train)

In [ ]:
# Evaluate the second level ensemble
accuracy_level_2 = second_level_ensemble.score(X_test_transformed, y_test)
print(f"Multi-level Ensemble of Ensembles Accuracy: {accuracy_level_2:.2f}")
# Predict using the second level ensemble
y_pred_level_2 = second_level_ensemble.predict(X_test_transformed)
print(classification_report(y_test, y_pred_level_2, target_names=class_names))

In this example, we first create a first-level ensemble using a voting classifier that combines predictions from a random forest, gradient boosting, and support vector machine. Then, we use the predictions from this first-level ensemble as input to a second-level stacking ensemble that combines it with logistic regression. This multi-level approach allows us to leverage the strengths of different ensemble methods at multiple levels, potentially improving predictive performance.